# 🌊 Marine Carbonate System - TA & pH

**Interactive Bjerrum Plot** using Total Alkalinity and pH parameters.

**Quick Instructions:**
- Adjust sliders to change TA and pH values
- Observe pH control effects on carbonate system
- Use preset buttons for different water types

In [ ]:
# Essential imports only
import PyCO2SYS as pyco2
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import FloatSlider, VBox, interactive_output, Button, HTML
from IPython.display import display

In [ ]:
# Simplified calculation function
def calc_system(alk, ph):
    """Calculate carbonate system from TA and pH"""
    results = pyco2.sys(par1=alk, par2=ph, par1_type=1, par2_type=3,
                       salinity=35, temperature=25, pressure=0,
                       opt_pH_scale=1, opt_k_carbonic=10)
    return {
        'pH': float(results['pH_total']),
        'pCO2': float(results['pCO2']),
        'HCO3': float(results['bicarbonate']),
        'CO3': float(results['carbonate']),
        'DIC': float(results['dic']),
        'omega': float(results['saturation_aragonite'])
    }

def plot_system(data):
    """Create simplified 4-panel plot"""
    plt.close('all')
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 8))
    
    # Bjerrum plot
    pH_range = np.linspace(5, 10, 30)
    H = 10**(-pH_range)
    K1, K2 = 1e-6, 1e-9  # Simplified constants
    denom = H**2 + K1*H + K1*K2
    
    ax1.plot(pH_range, H**2/denom, 'b-', linewidth=2, label='CO2')
    ax1.plot(pH_range, K1*H/denom, 'g-', linewidth=2, label='HCO3-')
    ax1.plot(pH_range, K1*K2/denom, 'r-', linewidth=2, label='CO3-2')
    ax1.axvline(data['pH'], color='orange', linestyle='--', linewidth=2)
    ax1.set_xlabel('pH'), ax1.set_ylabel('Fraction')
    ax1.set_title('Bjerrum Plot'), ax1.legend(), ax1.grid(True, alpha=0.3)
    
    # Composition pie
    CO2_conc = data['pCO2'] * 0.034
    species = [CO2_conc, data['HCO3'], data['CO3']]
    ax2.pie(species, labels=['CO2*', 'HCO3-', 'CO3-2'], autopct='%1.1f%%')
    ax2.set_title(f'Composition at pH {data["pH"]:.2f}')
    
    # Saturation
    omega = data['omega']
    color = 'green' if omega >= 1 else 'red'
    ax3.bar(['Aragonite'], [omega], color=color)
    ax3.axhline(1, color='black', linestyle='--')
    ax3.set_ylabel('Omega'), ax3.set_title('Saturation State')
    
    # Results
    ax4.axis('off')
    results_text = f"""pH = {data['pH']:.2f}
pCO2 = {data['pCO2']:.0f} μatm
HCO3- = {data['HCO3']:.0f} μmol/kg
CO3-2 = {data['CO3']:.0f} μmol/kg
DIC = {data['DIC']:.0f} μmol/kg
Ω_arag = {omega:.2f}

Conditions:
S = 35 PSU, T = 25°C"""
    ax4.text(0.1, 0.9, results_text, transform=ax4.transAxes, 
             fontsize=10, verticalalignment='top', fontfamily='monospace')
    
    plt.tight_layout()
    plt.show()
    return fig

In [ ]:
# Create interface
alk_slider = FloatSlider(value=2300, min=1800, max=3000, step=50,
                        description="Alkalinity:", style={'description_width': 'initial'})
ph_slider = FloatSlider(value=8.1, min=7.0, max=8.8, step=0.1,
                       description="pH:", style={'description_width': 'initial'})

def update_plot(alk, ph):
    """Update plot when sliders change"""
    data = calc_system(alk, ph)
    plot_system(data)

# Preset buttons
def reset_vals(b): alk_slider.value, ph_slider.value = 2300, 8.1
def ocean_vals(b): alk_slider.value, ph_slider.value = 2300, 8.1
def coastal_vals(b): alk_slider.value, ph_slider.value = 2400, 8.0
def acidified_vals(b): alk_slider.value, ph_slider.value = 2300, 7.6

reset_btn = Button(description="Reset", button_style='info')
ocean_btn = Button(description="Open Ocean", button_style='success')
coastal_btn = Button(description="Coastal", button_style='warning')
acid_btn = Button(description="Acidified", button_style='danger')

reset_btn.on_click(reset_vals)
ocean_btn.on_click(ocean_vals)
coastal_btn.on_click(coastal_vals)
acid_btn.on_click(acidified_vals)

# Display interface
display(VBox([
    HTML("<h3>🌊 pH Control Explorer</h3>"),
    alk_slider, ph_slider,
    VBox([ocean_btn, coastal_btn, acid_btn, reset_btn]),
    interactive_output(update_plot, {'alk': alk_slider, 'ph': ph_slider})
]))

print("✅ Fast pH control interface ready!")
print("🎯 Optimized for quick Binder loading")